In [1]:
# Import libraries
import random
import string

In [2]:
# Define variables
VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7
SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}
WORDLIST_FILENAME = "words.txt"

In [3]:
def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordList: list of strings
    wordList = []
    for line in inFile:
        wordList.append(line.strip().lower())
    print("  ", len(wordList), "words loaded.")
    return wordList

In [4]:
def getFrequencyDict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq

# Problem 1: Word Scores

The first step is to implement some code that allows us to calculate the score for a single word. The function getWordScore should accept as input a string of lowercase letters (a word) and return the integer score for that word, using the game's scoring rules.

In [5]:
def getWordScore(word, n):
    """
    Returns the score for a word. Assumes the word is a valid word.

    The score for a word is the sum of the points for letters in the
    word, multiplied by the length of the word, PLUS 50 points if all n
    letters are used on the first turn.

    Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is
    worth 3, D is worth 2, E is worth 1, and so on (see SCRABBLE_LETTER_VALUES)

    word: string (lowercase letters)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    returns: int >= 0
    """
    # Initialise a variable to store score
    score = 0
    
    # Calculate score for each letter
    for letter in word:
        score = score + SCRABBLE_LETTER_VALUES[letter]
        
    # Multiply by number of letters
    score = len(word) * score
    
    # Give extra 50 points if all letters are used
    if len(word) == n:
        score = score + 50
        
    return score

In [6]:
getWordScore('waybill', 7)

155

# Problem 2: Dealing with Hands

In [7]:
def dealHand(n):
    """
    Returns a random hand containing n lowercase letters.
    At least n/3 the letters in the hand should be VOWELS.

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    hand={}
    numVowels = n // 3

    for i in range(numVowels):
        x = VOWELS[random.randrange(0,len(VOWELS))]
        hand[x] = hand.get(x, 0) + 1
       
    for i in range(numVowels, n):
        x = CONSONANTS[random.randrange(0,len(CONSONANTS))]
        hand[x] = hand.get(x, 0) + 1
        
    return hand

In [8]:
hand = dealHand(HAND_SIZE)

In [9]:
def displayHand(hand):
    """
    Displays the letters currently in the hand.

    For example:
    >>> displayHand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    for letter in hand.keys():
        for j in range(hand[letter]):
             print(letter,end=" ")       # print all on the same line
    print()  

In [10]:
displayHand(hand)

i i d d d v b 


In [11]:
word = 'crop'

In [12]:
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    # Create a list of letters from hand
    hand_letter = list(hand.keys())
    
    # Create a dictionary out of word
    word_list = list(word)
#     word_dict = {}
#     for x in word_list:
#         word_dict[x] = word_dict.get(x,0) + 1
    word_dict = getFrequencyDict(word_list)

    # Find the difference between the dictionary of hand and dictionary of word
    values = []
    count = 0
    for key in hand.keys():
        values = values + [hand[key] - word_dict.get(key, 0)]
        count += 1
    result = dict(zip(hand_letter, values))
    new_hand = {key:val for key, val in result.items() if val > 0}
    
    return new_hand

In [13]:
updateHand(hand, word)

{'b': 1, 'd': 3, 'i': 2, 'v': 1}

# Problem 3: Valid words

A valid word is in the word list; and it is composed entirely of letters from the current hand. Implement the isValidWord function.

In [14]:
wordList = loadWords()

Loading word list from file...
   83667 words loaded.


In [15]:
def isValidWord(word, hand, wordList):
    """
    Returns True if word is in the wordList and is entirely
    composed of letters in the hand. Otherwise, returns False.

    Does not mutate hand or wordList.
   
    word: string
    hand: dictionary (string -> int)
    wordList: list of lowercase strings
    """
    
    # Create a dictionary out of word
    word_list = list(word)
    word_dict = {}
    for x in word_list:
        word_dict[x] = word_dict.get(x,0) + 1
        
    # Create a test variable
    test = []
    
    # Test if word consists of letters from hand
    for letter in word_dict.keys():
        if letter in hand.keys() and word_dict[letter] <= hand[letter]:
            test = test + [True]
        else:
            test = test + [False]
        
    # Test if word is included in wordlist
    if word in wordList:
        test = test + [True]
    else:
        test = test + [False]
        
    # Evaluate overall test
    test = all(test)
    
    return test

In [16]:
isValidWord(word, hand, wordList)

False

# Problem 4: Hand length

In [17]:
def calculateHandlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string int)
    returns: integer
    """
    length = 0
    for key in hand.keys():
        length = length + hand[key]
        
    return length

# Problem 5: Playing a Hand

In [18]:
def playHand(hand, wordList, n):
    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    * The user may input a word or a single period (the string ".") 
      to indicate they're done playing
    * Invalid words are rejected, and a message is displayed asking
      the user to choose another word until they enter a valid word or "."
    * When a valid word is entered, it uses up letters from the hand.
    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.
    * The sum of the word scores is displayed when the hand finishes.
    * The hand finishes when there are no more unused letters or the user
      inputs a "."

      hand: dictionary (string -> int)
      wordList: list of lowercase strings
      n: integer (HAND_SIZE; i.e., hand size required for additional points)
      
    """
    # BEGIN PSEUDOCODE <-- Remove this comment when you code this function; do your coding within the pseudocode (leaving those comments in-place!)
    # Keep track of the total score
    
    # As long as there are still letters left in the hand:
    total = 0
    score = 0
    while calculateHandlen(hand) > 0:
        print("Current Hand: ", end = "")

        # Display the hand
        displayHand(hand)

        # Ask user for input
        word = input("Enter word, or a '.' to indicate that you are finished: ")

        # Create a dictionary out of word
        word_list = list(word)
        word_dict = {}
        for x in word_list:
            word_dict[x] = word_dict.get(x,0) + 1

        # If input is '.'
        if word == '.':
            print("Goodbye! Total score: " + str(total) + " points.")
            print('')
            break

        # If the input is a single period:
        if word_dict == hand and isValidWord(word, hand, wordList) == True:
            score = getWordScore(word, n)
            total = total + score
            print('"' + word + '"' + " earned " + str(score) + " points. " + "Total: " + str(total) + " points.")
            print('')
            print("Run out of letters. Total score: " + str(total) + " points.")
            break
            # End the game (break out of the loop)

        # Otherwise (the input is not a single period):

        else:
            # If the word is not valid:
            if isValidWord(word, hand, wordList) == False:
                print("Invalid word, please try again.")
                print('')

            # If the word is valid:
            else:
                score = getWordScore(word, n)
                total = total + score
                print('"' + word + '"' + " earned " + str(score) + " points. " + "Total: " + str(total) + " points.")
                print('')
                hand = updateHand(hand, word)

In [19]:
playHand({'a': 2, 'e': 2, 'p': 1, 'r': 1, 't': 1}, wordList, 7)

Current Hand: a a e e p r t 
Enter word, or a '.' to indicate that you are finished: pear
"pear" earned 24 points. Total: 24 points.

Current Hand: a e t 
Enter word, or a '.' to indicate that you are finished: tea
"tea" earned 9 points. Total: 33 points.

Run out of letters. Total score: 33 points.


# Problem 6: Playing a game

A game consists of playing multiple hands. We need to implement one final function to complete our word-game program. Write the code that implements the playGame function. You should remove the code that is currently uncommented in the playGame body. Read through the specification and make sure you understand what this function accomplishes. For the game, you should use the HAND_SIZE constant to determine the number of cards in a hand.

In [20]:
def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.

    1) Asks the user to input 'n' or 'r' or 'e'.
      * If the user inputs 'n', let the user play a new (random) hand.
      * If the user inputs 'r', let the user play the last hand again.
      * If the user inputs 'e', exit the game.
      * If the user inputs anything else, tell them their input was invalid.
 
    2) When done playing the hand, repeat from step 1    
    """
    wordList = loadWords()
    hand = {}
    while True:
        text = input('Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
        if text == 'e':
            break

        elif text == 'n':
            hand = dealHand(HAND_SIZE)
            playHand(hand, wordList, HAND_SIZE)

        elif text == 'r' and hand == {}:
            print('You have not played a hand yet. Please play a new hand first!')
            print('')

        elif text == 'r' and hand != {}:
            playHand(hand, wordList, HAND_SIZE)

        else:
            print('Invalid command.')
            print('')

In [21]:
playGame(wordList)

Loading word list from file...
   83667 words loaded.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand: i e p t r l k 
Enter word, or a '.' to indicate that you are finished: trek
"trek" earned 32 points. Total: 32 points.

Current Hand: i p l 
Enter word, or a '.' to indicate that you are finished: pi
"pi" earned 8 points. Total: 40 points.

Current Hand: l 
Enter word, or a '.' to indicate that you are finished: .
Goodbye! Total score: 40 points.

Enter n to deal a new hand, r to replay the last hand, or e to end game: e


# Computer plays the game

Now that you have completed your word game code, you decide that you would like to enable your computer (SkyNet) to play the game (your hidden agenda is to prove once and for all that computers are inferior to human intellect!) In this part, you will be able to compare how you as a user succeed in the game compared to the computer's performance.

In [22]:
import time

In [23]:
# Computer chooses a word

def compChooseWord(hand, wordList, n):
    """
    Given a hand and a wordList, find the word that gives 
    the maximum value score, and return it.

    This word should be calculated by considering all the words
    in the wordList.

    If no words in the wordList can be made from the hand, return None.

    hand: dictionary (string -> int)
    wordList: list (string)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)

    returns: string or None
    """
    # Create a new variable to store the maximum score seen so far (initially 0)
    bestScore = 0
    # Create a new variable to store the best word seen so far (initially None)  
    bestWord = None
    # For each word in the wordList
    for word in wordList:
        # If you can construct the word from your hand
        if isValidWord(word, hand, wordList):
            # find out how much making that word is worth
            score = getWordScore(word, n)
            # If the score for that word is higher than your best score
            if (score > bestScore):
                # update your best score, and best word accordingly
                bestScore = score
                bestWord = word
    # return the best word you found.
    return bestWord

In [24]:
# Computer plays a hand

def compPlayHand(hand, wordList, n):
    """
    Allows the computer to play the given hand, following the same procedure
    as playHand, except instead of the user choosing a word, the computer 
    chooses it.

    1) The hand is displayed.
    2) The computer chooses a word.
    3) After every valid word: the word and the score for that word is 
    displayed, the remaining letters in the hand are displayed, and the 
    computer chooses another word.
    4)  The sum of the word scores is displayed when the hand finishes.
    5)  The hand finishes when the computer has exhausted its possible
    choices (i.e. compChooseWord returns None).
 
    hand: dictionary (string -> int)
    wordList: list (string)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    """
    # Keep track of the total score
    totalScore = 0
    # As long as there are still letters left in the hand:
    while (calculateHandlen(hand) > 0) :
        # Display the hand
        print("Current Hand: ", end=' ')
        displayHand(hand)
        # computer's word
        word = compChooseWord(hand, wordList, n)
        # If the input is a single period:
        if word == None:
            # End the game (break out of the loop)
            break
            
        # Otherwise (the input is not a single period):
        else :
            # If the word is not valid:
            if (not isValidWord(word, hand, wordList)) :
                print('This is a terrible error! I need to check my own code!')
                break
            # Otherwise (the word is valid):
            else :
                # Tell the user how many points the word earned, and the updated total score 
                score = getWordScore(word, n)
                totalScore += score
                print('"' + word + '" earned ' + str(score) + ' points. Total: ' + str(totalScore) + ' points')              
                # Update hand and show the updated hand to the user
                hand = updateHand(hand, word)
                print()
    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
    print('Total score: ' + str(totalScore) + ' points.')

In [25]:
def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.
 
    1) Asks the user to input 'n' or 'r' or 'e'.
        * If the user inputs 'e', immediately exit the game.
        * If the user inputs anything that's not 'n', 'r', or 'e', keep asking them again.

    2) Asks the user to input a 'u' or a 'c'.
        * If the user inputs anything that's not 'c' or 'u', keep asking them again.

    3) Switch functionality based on the above choices:
        * If the user inputted 'n', play a new (random) hand.
        * Else, if the user inputted 'r', play the last hand again.
          But if no hand was played, output "You have not played a hand yet. 
          Please play a new hand first!"
        
        * If the user inputted 'u', let the user play the game
          with the selected hand, using playHand.
        * If the user inputted 'c', let the computer play the 
          game with the selected hand, using compPlayHand.

    4) After the computer or user has played the hand, repeat from step 1

    wordList: list (string)
    """
    hand = {} 
    while True:
        
        # User input for type of hand
        text = input('Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
        print('')

        if text != 'n' and text != 'r' and text != 'e':
            print('Invalid command')

        # If the user wants to end the game
        elif text == 'e':
            break

        # If the user wants to repeat the game but hasn't started:
        elif text == 'r' and hand == {}:
            print('You have not played a hand yet. Please play a new hand first!')
            print('')

        # Otherwise
        else:

            # If the user wants to play a new game
            if text == 'n':
                hand = dealHand(HAND_SIZE)

            # If the user wants to play previous game
            else:
                hand = hand

            while True:
                turn = input('Enter u to have yourself play, c to have the computer play: ')
                print('')

                if turn != 'u' and turn != 'c':
                    print('Invalid command')
                    print('')

                # If user is human
                elif turn == 'u':
                    playHand(hand, wordList, HAND_SIZE)
                    break

                # If user is computer
                else:
                    compPlayHand(hand, wordList, HAND_SIZE)
                    break


In [26]:
playGame(wordList)

Enter n to deal a new hand, r to replay the last hand, or e to end game: n

Enter u to have yourself play, c to have the computer play: u

Current Hand: i e s s g z w 
Enter word, or a '.' to indicate that you are finished: wiz
"wiz" earned 45 points. Total: 45 points.

Current Hand: e s s g 
Enter word, or a '.' to indicate that you are finished: eg
Invalid word, please try again.

Current Hand: e s s g 
Enter word, or a '.' to indicate that you are finished: seg
"seg" earned 12 points. Total: 57 points.

Current Hand: s 
Enter word, or a '.' to indicate that you are finished: .
Goodbye! Total score: 57 points.

Enter n to deal a new hand, r to replay the last hand, or e to end game: r

Enter u to have yourself play, c to have the computer play: c

Current Hand:  i e s s g z w 
"wizes" earned 85 points. Total: 85 points

Current Hand:  s g 
Total score: 85 points.
Enter n to deal a new hand, r to replay the last hand, or e to end game: e

